### Initialization
* Check whether the runtime is host or local.
* Mount Google Drive when using the host runtime.

In [0]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
  runtime = "host"
except:
  runtime = "local"

### Parameters

In [0]:
#@title Parameters
#@markdown |Name            |Description|
#@markdown |:---            |:---|
#@markdown |`seed`|The random seed|
seed = 3984 #@param {type: "number"}

#@markdown ### `deep-coder` Repositories
#@markdown |Name            |Description|
#@markdown |:---            |:---|
#@markdown |`repository_url`|The URL of `deep-coder` git repository (enabled only in the host runtime)|
#@markdown |`branch_name`   |The branch name (enabled only in the host runtime)|
repository_url = "https://github.com/HiroakiMikami/deep-coder" #@param {type: "string"}
branch_name = "master" #@param {type: "string"}

#@markdown ### Filepathes
#@markdown |Name                |Description|
#@markdown |:---                |:---|
#@markdown |`valid_dataset_path`|The file path of the validation dataset|
#@markdown |`result_pathes`     |The comma separated list of the validation results|
valid_dataset_path = "dataset/valid.pickle" #@param {type: "string"}
result_pathes = "out/baseline/result.pickle,out/deep-coder/result.pickle" #@param {type: "string"}
result_path_list = list(result_pathes.split(","))

### Setup
* Fix the random seed
* Download the codebase (when using the host runtime)
  1. Clone git repository and move to the specified branch
  2. Initialize submodule
  3. Build the `search` tool
  4. Install chainer and cupy

In [0]:
import numpy as np
import random

SEED_MAX = 2**32 - 1

root_rng = np.random.RandomState(seed)
random.seed(root_rng.randint(SEED_MAX))
np.random.seed(root_rng.randint(SEED_MAX))

In [0]:
if runtime == "host":
  %cd /content
  !rm -rf deep-coder
  ![ ! -e deep-coder ] && git clone $repository_url deep-coder
  %cd deep-coder
  !git checkout origin/$branch_name
  !git submodule init
  !git submodule update
  !make -C DeepCoder_Utils/enumerative-search -j `nproc`
  !curl https://colab.chainer.org/install | sh -

### Load Dataset and Results
* Load dataset
* Load results

In [0]:
import pickle

# Load model
with open(valid_dataset_path, "rb") as f:
    valid_dataset = pickle.load(f)

# Load results
results = dict()
for path in result_path_list:
    with open(path, "rb") as f:
        results[path] = pickle.load(f)


### Visualize Results


* Show the time needed to solve

In [0]:
#@markdown ### Visualization Parameters
#@markdown |Name    |Description|
#@markdown |:---    |:---|
#@markdown |`width` |The width of the matplotlib plot|
#@markdown |`height`|The height of the matplotlib plot|
width = 12 #@param {type: "slider", min: 1, max: 48, step: 1}
height = 4 #@param {type: "slider", min: 1, max: 48, step: 1}


In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(width, height))

required_times = dict()
for path in results.keys():
    t = [r.time_seconds for r in results[path].values() if r.is_solved]
    t.sort()
    required_times[path] = t

plt.axis("tight")
plt.axis("off")
plt.title("Search Speed")

rowLabels = list(results.keys())
colLabels = ["20%", "40%", "60%", "80%", "100%"]

text = []
for path in rowLabels:
    r = 0.0
    t = required_times[path]
    row = []
    for i in range(5):
        r += 0.2
        num = int(len(valid_dataset.dataset) * r)
        row.append("{} sec".format(t[num]) if len(t) > num else "-")
    text.append(row)

plt.table(cellText=text, colLabels=colLabels,
               rowLabels=rowLabels, loc="center")

* Show the nodes exploration needed to solve

In [0]:
#@markdown ### Visualization Parameters
#@markdown |Name    |Description|
#@markdown |:---    |:---|
#@markdown |`width` |The width of the matplotlib plot|
#@markdown |`height`|The height of the matplotlib plot|
width = 12 #@param {type: "slider", min: 1, max: 48, step: 1}
height = 4 #@param {type: "slider", min: 1, max: 48, step: 1}


In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(width, height))

required_nodes = dict()
for path in results.keys():
    n = [r.explored_nodes for r in results[path].values() if r.is_solved]
    n.sort()
    required_nodes[path] = n

plt.axis("tight")
plt.axis("off")
plt.title("Explored Nodes")

rowLabels = list(results.keys())
colLabels = ["20%", "40%", "60%", "80%", "100%"]

text = []
for path in rowLabels:
    r = 0.0
    t = required_nodes[path]
    row = []
    for i in range(5):
        r += 0.2
        num = int(len(valid_dataset.dataset) * r)
        row.append("{}".format(t[num]) if len(t) > num else "-")
    text.append(row)

plt.table(cellText=text, colLabels=colLabels,
               rowLabels=rowLabels, loc="center")

* Show the detail of the specified entry

In [0]:
#@markdown ### Visualization Parameters
#@markdown |Name    |Description|
#@markdown |:---    |:---|
#@markdown |`width` |The width of the matplotlib plot|
#@markdown |`height`|The height of the matplotlib plot|
#@markdown |`index`  |The index of the specified entry|
width = 36 #@param {type: "slider", min: 1, max: 48, step: 1}
height = 4 #@param {type: "slider", min: 1, max: 48, step: 1}
index = 0 #@param {type: "number"}


In [0]:
from matplotlib import colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt

m = cm.ScalarMappable(norm=colors.Normalize(vmin=0, vmax=1), cmap=cm.Greens)

plt.figure(figsize=(width, height))
    
entry = valid_dataset.dataset[index][0]
print("Valid {}".format(index))
print("Examples")
for i, example in enumerate(entry.examples):
    print("Example {}".format(i))
    for j, input in enumerate(example.inputs):
        print("  input {}: {}".format(j, input))
    print("  output: {}".format(example.output))
print()

print("Source Code")
print("Ground Truth")
print(entry.source_code)
print()
for path, result in results.items():
    print(path)
    print(result[index].solution)
    print()

plt.title("Attributes")
labels = ["Ground Truth"]
ticks = [0.5]
data = np.ones(len(entry.attribute))
colors = []
for _, v in entry.attribute.items():
    colors.append(m.to_rgba(1 if v else 0))
xs = np.arange(len(entry.attribute))
plt.bar(xs, data, width=0.9, bottom=np.zeros(1),
        color=colors, tick_label=list(entry.attribute.keys()))

for i, (path, result) in enumerate(results.items()):
    labels.append(path)
    ticks.append(i + 1.5)
    colors = []
    for key in entry.attribute.keys():
        colors.append(m.to_rgba(result[index].probabilities[key]))
    plt.bar(xs, data, width=0.9, bottom=np.ones(1) * (i + 1), color=colors,
            tick_label=list(entry.attribute.keys()))

plt.yticks(ticks=ticks, labels=labels)
